# Incompressible Navier-Stokes equations: Coral formulation

## Differential formulation

We consider the incompressible Navier-Stokes equations, in the velocity/pressure formulation:
\begin{gather}
\frac{\partial}{\partial t} \mathbf{u}= - \mathbf{\nabla} p - \mathbf{u\cdot \nabla u} + \mathrm{Re}^{-1} \Delta \mathbf{u} \\
\mathbf{\nabla \cdot u} = 0
\end{gather}
where $\Delta=\partial^2_{xx}+\partial^2_{yy}+\partial^2_{zz}$ is the Laplacian and $\mathrm{Re}$ is the Reynolds number.  Note that using incompressibility, the advective term in the conservation of momentum equation can be written as a divergence:
\begin{equation}
\frac{\partial}{\partial t} \mathbf{u}= - \mathbf{\nabla} p -  \mathbf{\nabla \cdot} \left( \mathbf{u} \otimes \mathbf{u}\right) + \mathrm{Re}^{-1} \Delta \mathbf{u}
\end{equation}
Anticipating that the horizontal and the vertical directions are inherently different from the quasi-inverse viewpoint, we introduce the horizontal laplacian 
\begin{equation}
\nabla^2_\perp = \partial_{xx} + \partial_{yy}
\end{equation}

* * *
## Quasi-Inverse formulation

### Horizontal velocity equations: canonical QI procedure

We first apply the QI procedure to the governing equations for the horizontal velocities $u$ and $v$. For simplicity, we limit ourselves to the linear dynamics:
\begin{gather}
\frac{\partial}{\partial t} u = - \partial_x p + \mathrm{Re}^{-1}\partial_{zz}u + \mathrm{Re}^{-1}\nabla^2_\perp u \\
\frac{\partial}{\partial t} v = - \partial_y p + \mathrm{Re}^{-1}\partial_{zz}v + \mathrm{Re}^{-1} \nabla^2_\perp v
\end{gather}
Upon integrating the governing equations for $u$ and $v$ twice with respect to $z$, one gets
\begin{gather}
\frac{\partial}{\partial t} \iint u = - \iint \partial_x p + \mathrm{Re}^{-1} u + \mathrm{Re}^{-1} \iint \nabla^2_\perp u \\
\frac{\partial}{\partial t} \iint v = - \iint \partial_y p + \mathrm{Re}^{-1} v + \mathrm{Re}^{-1} \iint \nabla^2_\perp v
\end{gather}
Observe that the straightforward, canonical QI procedure that was illustrated before on the heat equations has no reason to fail here: $u$ and $v$ are both governed by 2nd-order (in z) differential equations, each of which can easily be supplied with 2 boundary conditions (no-slip or stress-free for top $z=1$ and bottom $z=0$).

### Vertical velocity: the thorny issue of incompressibility
Upon inspection, things are different for the vertical velocity. Consider the remaining equations:
\begin{gather}
\frac{\partial}{\partial t} w = - \partial_z p + \mathrm{Re}^{-1}\partial_{zz}w + \mathrm{Re}^{-1}\nabla^2_\perp w\\
0 = \partial_x u + \partial_y v + \partial_z w 
\end{gather}

These equations are of order $2$ and $1$. At the time of writting, Coral is not designed for imposing an odd number of boundary conditions. For this reason, we introduce the auxiliary variable 
$\chi=\partial_z w$ and rewrite the equations above as:
\begin{gather}
\frac{\partial}{\partial t} w = - \partial_z p + \mathrm{Re}^{-1}\partial_{z}\chi +\mathrm{Re}^{-1} \nabla^2_\perp w\\
0 = \chi - \partial_z w\\
0 = \partial_x u + \partial_y v + \chi 
\end{gather}
The reformulated equations are of order $1$, $1$, and $0$. This time, they can be supplemented with regular impenetrable boundary conditions $w=0$ at $z=0,1$ (with no need for additional boundary conditions). We can now formally integrate each equation to obtain:
\begin{gather}
\frac{\partial}{\partial t} \int w = - p + \mathrm{Re}^{-1}\chi + \mathrm{Re}^{-1}\int \nabla^2_\perp w\\
0 = \int\chi - w\\
0 = \partial_x u + \partial_y v + \chi 
\end{gather}

### Nonlinear QI equations

Now that the linear equations are ready, it is straightforward to include fluid advection, so that the full set of equations becomes, for the $(k_x,k_y)\ne (0,0)$ mode:

\begin{gather}
\frac{\partial}{\partial t} \iint u = - \iint \partial_x p + \mathrm{Re}^{-1} u + \mathrm{Re}^{-1} \iint \nabla^2_\perp u -\iint u^2 - \iint uv -\int uw\\
\frac{\partial}{\partial t} \iint v = - \iint \partial_y p + \mathrm{Re}^{-1} v + \mathrm{Re}^{-1} \iint \nabla^2_\perp v -\iint uv - \iint v^2 -\int vw\\
\frac{\partial}{\partial t} \int w = - p + \mathrm{Re}^{-1}\chi + \mathrm{Re}^{-1}\int \nabla^2_\perp w - \int uw -\int vw - w^2 \\
0 = \int\chi - w\\
0 = \partial_x u + \partial_y v + \chi 
\end{gather}

For the horizontal mean ($k_x=k_y=0$ mode), one gets:
\begin{gather}
\frac{\partial}{\partial t} \iint u = \mathrm{Re}^{-1} u -\int uw\\
\frac{\partial}{\partial t} \iint v = \mathrm{Re}^{-1} v -\int vw
\end{gather}
* * * 
# Formating the equations for Coral

## List of Parameters

We have only one parameter, the inverse of the Reynolds number. We denote this parameter `iRe`. For instance, if we consider $\mathrm{Re}=500$, we add at the top of `coral.equations`:
```
>>Param :: iRe :=0.002 <<
```

## List of variables
### Overview and a simple example
Coral require that the user define four kind of variables:
+ `>>linear_variable_kxky` are fluctuating components of fields, that are linearly coupled
+ `>>linear_variable_mean` are horizontal-mean components of fields (i.e. functions of the vertical coordinate `z` only), that are linearly coupled
+ `>>linear_variable_full` are linear combinations of fluctuating and / or horizontal-mean components. (That is to say that they are necessarily formed by combining members from `>>linear_variable_kxky` and `>>linear_variable_zero`.) Their reason is being is to be associated in quadratic variables (see next bullet).
+ `>>quadratic_variable` are products of `>>linear_variable_full` and may represent advection.

Before we move to turning our Navier-Stokes problem into a Coral input file, we first consider a simpler equation that contains only one field $u$:
\begin{equation}
\partial_t u = \partial_{zz} u + \partial_{yy}u + \partial_{xx} u+ \partial_z u^2
\end{equation}
For Coral, the horizontal-average $\overline{u}(z)=\int dx \int dy\: u(x,y,z)$ and the fluctuations around this mean $\widetilde{u}(x,y,z)=u(x,y,z) - \overline{u}(z)$ are treated as distinct linear variables, and denoted `uMean` and `u`. The full field $u$, denoted `uFull`, is recovered by computing the sum `uMean + u`. Finally, the quadratic variable `u2` represents the square of `uFull`. Hence the declaration of variables:
```
>>====================================<<
>>linear_variable_kxky :: uk <<
>>linear_variable_mean :: u0 <<
>>====================================<<
>>linear_variable_full       :: uFull <<
>>linear_variable_full_build :: + uMean <<
>>linear_variable_full_build :: + u <<
>>====================================<<
>>quadratic_variable :: uu := uFull.uFull <<
>>====================================<<
```
### Variables for the Navier-Stokes equations
We merely need to reproduce the pattern described above with the Navier-Stokes equations. 

Starting with the horizontal-means, we denote `uMean` and `vMean` the horizontal mean flows. 

Concerning the fluctuating components, we denote `u` and `v` the fluctuating components of the horizontal velocities. `w`, `chi`, and `p` denote the ($k_x\ne 0$ or $k_y\ne 0$) vertical velocity, vertical velocity derivative, and pressure, respectively. 

For quadratic term, we need the three components of velocity, defined as `uFull`, `vFull`, and `wFull`. (Observe that, although `wFull = w`, these variables have to be declared separately, as they correspond to different objects.)

Finally, the fields defined above are combined into `uu`, `uv`, `uw`, `vv`, `vw`, and `ww`.

```
>>====================================<<
>>linear_variable_kxky :: u <<
>>linear_variable_kxky :: v <<
>>linear_variable_kxky :: w <<
>>linear_variable_kxky :: p <<
>>linear_variable_kxky :: chi <<
>>linear_variable_mean :: uMean <<
>>linear_variable_mean :: vMean <<
>>====================================<<
>>linear_variable_full       :: uFull <<
>>linear_variable_full_build :: + uMean <<
>>linear_variable_full_build :: + u <<
>>====================================<<
>>linear_variable_full       :: vFull <<
>>linear_variable_full_build :: + vMean <<
>>linear_variable_full_build :: + v <<
>>====================================<<
>>linear_variable_full       :: wFull <<
>>linear_variable_full_build :: + w <<
>>====================================<<
>>quadratic_variable :: uu := uFull.uFull <<
>>quadratic_variable :: uv := uFull.vFull <<
>>quadratic_variable :: uw := uFull.wFull <<
>>quadratic_variable :: vv := vFull.vFull <<
>>quadratic_variable :: vw := vFull.wFull <<
>>quadratic_variable :: ww := wFull.wFull <<
>>====================================<<
```